In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import model_from_json, clone_model

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
#Clear the previous tensorflow graph
K.clear_session()

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

dataFolder = '../CMAPSSData'

datasets = [1]
iterations = 10
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
#windowSizes = {1:25,2:15,3:25,4:15}
windowSizes = 15
stride = 2
#constRUL = {1:93, 2:94, 3:92, 4:94}
constRUL = 95
#models = {1:RULmodel_SN_1,2:RULmodel_SN_2,3:RULmodel_SN_3,4:RULmodel_SN_4}
models = {1:RULmodel_SN_1}

#Create and compile the model
windowSize = 15
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.loadData()
tModel.epochs = 50

#file = open("ResultsBestModelAllDatasetsSingleSet.csv", "w")
#csvfile = csv.writer(file, lineterminator='\n')

min_rmse = 1000
bestModel = None
bestIndex = 0

tempScoresRMSE = []
tempScoresRHS = []
tempTime = []

bestModel = clone_model(tModel.model)

for i in range(iterations):

    print("Iteration "+str(i+1))
    #file.write("Iteration "+str(j+1)+'\n\n')
    tModel.trainModel(learningRateScheduler=lrate, verbose=0)
    tModel.evaluateModel(["rhs"], round=True)

    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rmse2 = cScores['rmse']
    rhs = cScores['rhs']
    time = tModel.trainTime

    if rmse2 < min_rmse:
        bestModel.set_weights(tModel.model.get_weights())
        bestIndex = i
        min_rmse = rmse2
    
    tempScoresRMSE.append(rmse2)
    tempScoresRHS.append(rhs)
    tempTime.append(time)

print(tempScoresRMSE)
print('Best Score')
print(min_rmse)
    
#save best model to file
# serialize model to JSON
model_json = bestModel.to_json()
with open("bestRULModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
bestModel.save_weights("bestRULModel.h5")
print("Saved model to disk")
        
#file.close()

Iteration 1
100/100 [==============================] - 0s 276us/step
Iteration 2
100/100 [==============================] - 0s 40us/step
Iteration 3
100/100 [==============================] - 0s 70us/step
Iteration 4
100/100 [==============================] - 0s 40us/step
Iteration 5
100/100 [==============================] - 0s 40us/step
Iteration 6
100/100 [==============================] - 0s 30us/step
Iteration 7
100/100 [==============================] - 0s 40us/step
Iteration 8
100/100 [==============================] - 0s 40us/step
Iteration 9
100/100 [==============================] - 0s 40us/step
Iteration 10
100/100 [==============================] - 0s 40us/step
[14.647866738880444, 13.220060514233662, 12.7534309109353, 13.467367968537877, 13.621306838919677, 12.859237924542807, 14.361058456812994, 13.521834195108296, 13.204166009256321, 13.465140177510222]
Best Score
12.7534309109353
Saved model to disk


In [27]:
# load json and create model
json_file = open('bestRULModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("bestRULModel.h5")
print("Loaded model from disk")


#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

windowSize = 15
stride = 2
constRUL = 95


dataFolder = '../CMAPSSData'

#Create and compile the models
#nFeatures = len(selected_features)
#shapeSN = nFeatures*windowSize
#modelRULSN = RULmodel_SN_1(shapeSN)
loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('Loaded_Model', loaded_model, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.loadData(verbose=1)
tModel.getModelDescription()



Loaded model from disk
Loading data for dataset 1 with window_size of 15, stride of 2 and constRUL of 95. Cros-Validation ratio 0
Data loaded for dataset 1
Description for model: Loaded_Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 30)                6330      
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
fc2 (Dense)                  (None, 10)                310       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
out (Dense)                  (None, 1)                 11        
Total params: 6,651
Trainable params: 6,651
Non-trainable params: 0
______________

In [28]:
# evaluate loaded model on test data
tModel.evaluateModel(["rhs"], round=True)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 531us/step
scores
{'rmse': 12.7534309109353, 'rhs': 258.3132150863941, 'score_1': 163.44337829589844, 'loss': 163.44337829589844}
RMSE: 12.784497576983558
RMSE2: 12.7534309109353
RHS: 258.3132150863941
Time : None seconds
